# Import Packages

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tabulate import tabulate

# repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load the raw data

In [2]:
demographics_problems_merged = pd.read_pickle('../data/clean_presented_problems.pkl')

## Explore the data

In [3]:
demographics_problems_merged.head()

,Local ID,state_demographics,enrollment_date,enrollment_ym_derived,enrollment_year,disability_level,disability_level_ordered,presented_problems,Gender,Ethnicity,County,is_aggression
0,000083W,north carolina,2018-06-27,2018-06-01,2018,Mild,Mild,"aggression (physical, verbal, property destruc...",Female,Not of Hispanic origin,Swain,1
1,000083W,north carolina,2018-06-27,2018-06-01,2018,Mild,Mild,decrease in ability to participate in daily fu...,Female,Not of Hispanic origin,Swain,0
2,000083W,north carolina,2018-06-27,2018-06-01,2018,Mild,Mild,leaving unexpectedly,Female,Not of Hispanic origin,Swain,0
3,000083W,north carolina,2018-06-27,2018-06-01,2018,Mild,Mild,mental health symptoms,Female,Not of Hispanic origin,Swain,0
4,1021487,texas,2020-03-02,2020-03-01,2020,Mild,Mild,"aggression (physical, verbal, property destruc...",Male,Not of Hispanic origin,Tarrant,1


## Define function

In [5]:
def save_model_output(model, i):
    # Extract the coefficient table from the model summary and format it nicely
    coef_table = model.params.to_frame()
    coef_table.columns = ['Coefficient']
    coef_table['Standard Error'] = model.bse
    coef_table['z-value'] = model.tvalues
    coef_table['P-value'] = model.pvalues
    coef_table = coef_table.apply(pd.to_numeric)

    # Save the formatted coefficient table to a text file
    with open(f"../output/logistic_regression_output{i}.txt", "w") as f:
        f.write(tabulate(coef_table, headers='keys', tablefmt='fancy_grid'))

In [6]:
## Perform a logistic regression to see if there is a relationship between disability level and aggression
model = smf.logit("is_aggression ~ C(disability_level_ordered, Treatment('Normal intelligence'))", data = demographics_problems_merged).fit()

print(model.summary())
save_model_output(model, 0)

Optimization terminated successfully.
         Current function value: 0.549546
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          is_aggression   No. Observations:                16406
Model:                          Logit   Df Residuals:                    16400
Method:                           MLE   Df Model:                            5
Date:                Mon, 13 Mar 2023   Pseudo R-squ.:                0.001580
Time:                        22:15:28   Log-Likelihood:                -9015.9
converged:                       True   LL-Null:                       -9030.1
Covariance Type:            nonrobust   LLR p-value:                 2.869e-05
                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------

In [7]:
## Perform a logistic regression to see if there is a relationship between disability level, gender, and state 
model = smf.logit("is_aggression ~ C(disability_level_ordered, Treatment('Normal intelligence')) + C(state_demographics, Treatment('new york'))", data = demographics_problems_merged).fit()

print(model.summary())

Optimization terminated successfully.
         Current function value: 0.547205
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          is_aggression   No. Observations:                16406
Model:                          Logit   Df Residuals:                    16394
Method:                           MLE   Df Model:                           11
Date:                Mon, 13 Mar 2023   Pseudo R-squ.:                0.005833
Time:                        22:15:29   Log-Likelihood:                -8977.4
converged:                       True   LL-Null:                       -9030.1
Covariance Type:            nonrobust   LLR p-value:                 1.553e-17
                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------

In [8]:
## Perform a logistic regression to see if there is a relationship between disability level, gender, and state 
model = smf.logit("is_aggression ~ C(disability_level_ordered, Treatment('Normal intelligence')) + C(Gender, Treatment('Male')) + C(state_demographics, Treatment('new york'))", data = demographics_problems_merged).fit()

print(model.summary())
save_model_output(model, 1)

Optimization terminated successfully.
         Current function value: 0.546977
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          is_aggression   No. Observations:                16406
Model:                          Logit   Df Residuals:                    16393
Method:                           MLE   Df Model:                           12
Date:                Mon, 13 Mar 2023   Pseudo R-squ.:                0.006248
Time:                        22:15:29   Log-Likelihood:                -8973.7
converged:                       True   LL-Null:                       -9030.1
Covariance Type:            nonrobust   LLR p-value:                 1.644e-18
                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------